### Map CAD instantiedata rioleringen Utrecht naar NLCS Classificatie concepten obv NLCS Tekenstandaard laagnamen

In [ ]:
import json
from rdflib import Graph

with open("resources/points-utrecht.geojson", 'r') as f:
    data = json.load(f)

kopieOriginalFile = {
    "type": "FeatureCollection",
    "name": "points-utrecht-alignment-nlcs",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:EPSG::7415"
        }
    },
    "features": []
}

g = Graph().parse("resources/NLCS_Classificatie_Kern_v0_0_4.ttl")

In [ ]:
def retrieve_related_nlcs_class(g, name):
  nlcsLayerQuery = f"""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
  PREFIX dcterms: <http://purl.org/dc/terms/>
  PREFIX sh: <http://www.w3.org/ns/shacl#>

  SELECT ?uri
  WHERE {{
    ?uriRi a owl:Class .
    ?uriRi skos:prefLabel "RI" .
    ?uri rdfs:subClassOf+ ?uriRi .
    ?uri skos:prefLabel "{name}" .
  }} LIMIT 1
  """

  results = g.query(nlcsLayerQuery)
  for res in results:
    return res.uri

In [ ]:
features = data['features']

nlcs = []
nlcsClean = []
featuresNlcs = []
for feature in features:
    properties = feature['properties']
    if properties['autocad_Z_layer']:
        if '-RI-' in properties['autocad_Z_layer']:
            # print(properties['autocad_Z_layer'])
            name = properties['autocad_Z_layer'].split("-RI-")[1].split("-")[0]
            # name = name.replace("VWA_", "DWA_")  # is VWA gelijk aan DWA? DWA heeft external link naar gwsw:VuilWaterStelsel > Elisabeth denkt van wel
            uri = retrieve_related_nlcs_class(g, name)
            feature["nlcs_classificatie_object"] = uri
            featuresNlcs.append(feature)
            nlcs.append(properties['autocad_Z_layer'])
            nlcsClean.append(name)

nlcs = list(set(nlcs))
for layer in nlcs:
    print(layer)


R-WE-RI-HWA_TRANSPORTLEIDING_PVC_125-INLAATKOLK-S
B-WE-RI-VWA_RIOOLPUT-S
R-WE-RI-HWA_RIOOLPUT-S
R-WE-RI-VWA_TRANSPORTLEIDING_PVC_160_VERLOOP_125_FASE_BLIND-S
R-WE-RI-VWA_HULPSTUK_EINDKAP_160-S
R-WE-RI-HWA_TRANSPORTLEIDING_PVC_125-INLAAT-AFGEDOPT-S
R-WE-RI-VWA_HULPSTUK_ERFSCHEIDINGSPUT-125_FASE_RESTANT-S
R-WE-RI-VWA_RIOOLPUT-S
R-WE-RI-VWA_VETAFSCHEIDER-RIOOLPUT-S
R-WE-RI-HWA_UITSTROOMBAK-S
R-WE-RI-HWA_TRANSPORTLEIDING_VERVAL-S
R-WE-RI-HWA_TRANSPORTLEIDING_PVC_125-INLAAT-S
R-WE-RI-VWA_TRANSPORTLEIDING_PVC_125-INLAAT-S
R-WE-RI-HWA_HULPSTUK_ERFSCHEIDINGSPUT-125_FASE_RESTANT-S


In [8]:
kopieOriginalFile["features"] = featuresNlcs

json_str = json.dumps(kopieOriginalFile)  #, indent=4)
with open("resources/points-utrecht-nlcs.geojson", "w") as f:
    f.write(json_str)
